# Overview

Standard checks / results presentation.

It loads a parquet data frame that has been created in a different script.  You can also load that data into the viewer script, which has a search interface.

This is more about having a standard set of tables / charts that are top level checks to look through.

Originally it was at the end of the main script.

# Imports

In [ ]:
# Safe imports
from datetime import datetime, timedelta, date

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import ipywidgets as widgets

In [ ]:
# Imports on files that might have dependencies that need installing
import data_pier_querying
from athena_querying import AthenaQuery
from athena_common_queries import *
#import user_agents # this converts user agent from browser to mobile / desktop etc.

# Settings & Loading Data

In [ ]:
# load_merged_resutls_file_path = "merged_df_with_meta_01-10_01-12.gzip"
<<<<<<< local
load_merged_results_file_path = "merged_df_with_meta_20200216_0000_to_20200223_0000.gzip"
=======
load_merged_results_file_path = "merged_df_with_meta_20200209_0000_to_20200216_0000.gzip"
>>>>>>> remote
just_sg_hk = True # a little bit dangerous as you'll also miss out ones without country set.

In [ ]:
merged_df_with_meta = pd.read_parquet(load_merged_results_file_path)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
pyarrow or fastparquet is required for parquet support

In [ ]:
if just_sg_hk:
    merged_df_with_meta = merged_df_with_meta[merged_df_with_meta.country_code.isin(["sg", "hk"])]

NameError: name 'merged_df_with_meta' is not defined

In [ ]:
merged_df_with_meta.head()

<<<<<<< local


,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code,user_agent,device_family,os_family,os_version,browser_family,browser_version,is_bot
0,00000487-4c71-4c7e-a372-ab7196780fb0,PageView,https://blog.moneysmart.hk/zh-hk/credit-cards/...,2020-02-18,1,1,blog,/zh-hk/credit-cards/%e9%85%92%e5%ba%97%e8%87%a...,/credit-cards,control,hk,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,desktop,Windows,7,Chrome,80.0.3987,False
1,00000487-4c71-4c7e-a372-ab7196780fb0,Reading,https://blog.moneysmart.hk/zh-hk/credit-cards/...,2020-02-18,3,3,blog,/zh-hk/credit-cards/%e9%85%92%e5%ba%97%e8%87%a...,/credit-cards,control,hk,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,desktop,Windows,7,Chrome,80.0.3987,False
2,000004a5-230d-4d4c-a850-d2ec44427589,PageView,https://www.moneysmart.sg/credit-cards/posb-ev...,2020-02-21,1,1,shop,/credit-cards/posb-everyday-card,/credit-cards,control,sg,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like...,mobile,iOS,13.3.1,Mobile Safari,13.0.5,False
3,00000bc0-99d3-4742-a855-74d49f6b617c,PageView,https://www.moneysmart.sg/embed/f645886bc03619...,2020-02-18,1,1,shop,/embed/f645886bc036195148acd846a50232d9,/embed,control,sg,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,desktop,Windows,10,Chrome,79.0.3945,False
4,00000bc0-99d3-4742-a855-74d49f6b617c,UserView.WidgetLoad,https://www.moneysmart.sg/embed/f645886bc03619...,2020-02-18,1,1,shop,/embed/f645886bc036195148acd846a50232d9,/embed,control,sg,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,desktop,Windows,10,Chrome,79.0.3945,False


NameError: name 'merged_df_with_meta' is not defined

>>>>>>> remote


In [ ]:
len(merged_df_with_meta)

<<<<<<< local <modified: text/plain>


2008810

1794946

>>>>>>> remote <modified: text/plain>


In [ ]:
merged_df_with_meta.dtypes

anonymous_id         object
event_name           object
page_url             object
date               category
s_count               int64
k_count               int64
page_type          category
slug               category
slug_root            object
ab_test            category
country_code       category
user_agent           object
device_family      category
os_family          category
os_version         category
browser_family     category
browser_version    category
is_bot               object
dtype: object

# Common Code

In [ ]:
def colour_grouped_table(df):

    def color_how_good(value):
        if isinstance(value, str):
            return
        av = abs(value)
        if av<2:
            c = "green"
        elif value <0:
            c =  "red"

        else:
            c= "blue"

        if av>20:
            return "background-color:rgb(250,200,200)"
        return "color:%s"%c # it's just CSS, so you can do background as well.
    return df.style.applymap(color_how_good , subset=["k_vs_s_%"])

In [ ]:
def group_by_and_show_count_difference(df, group_by_cols, with_styling=True):
    """
    This expects counts in s_count and k_count
    """
    
    grouped = df.groupby(group_by_cols, observed=True).sum().reset_index() #observed prevents stupid inefficiency with category columns
    
    grouped["k_vs_s_%"] = grouped.apply(lambda row:(999 if row.k_count else 0) if row.s_count==0 else round(((row.k_count - row.s_count)/row.s_count)*100, 1), axis=1 )
    grouped = grouped[(grouped.k_count>0) | (grouped.s_count>0)] # filters out NaNs after grouping
    
    

    return grouped

# Top Level Checks

In [ ]:
def group_by_and_show_count_difference(df, group_by_cols, with_styling=True):
    """
    This expects counts in s_count and k_count
    """
    
    #TODO: would be even better to aggregate using count as well as sum (and I think count not size) to remove duplicates, and highlight on that as well.
    
<<<<<<< local
    grouped = df.groupby(group_by_cols).sum().reset_index()
        
=======
>>>>>>> remote
    grouped["k_vs_s_%"] = grouped.apply(lambda row:(999 if row.k_count else 0) if row.s_count==0 else round(((row.k_count - row.s_count)/row.s_count)*100, 1), axis=1 ) #TODO: likely a lot faster using pandas functions not lambda or numpy compile
    grouped = grouped[(grouped.k_count>0) | (grouped.s_count>0)] # filters out NaNs after grouping
    
    

    return grouped

In [ ]:

def highlight_cols(cell):
    #use hex colours, or named ones to ensure excel compatibility on export
    if cell=="":
        return ""
    ci = min(100, int(abs(cell*10)))
    if abs(cell)<=2:
        return "color:green;"
    if cell <0:

        return "background-color:#%02x%02x%02x;" % (255,255-ci,255-ci)
    if cell>0:
        #return "background-color:cyan;"
        return "background-color:#%02x%02x%02x;" % (255-ci, 255-ci, 255)

    return "background-color:red;"


def color_how_good(value): # old one, might be junk
    if isinstance(value, str):
        return
    av = abs(value)
    if av<2:
        c = "green"
    elif value <0:
        c =  "red"

    else:
        c= "blue"

    if av>20:
        return "background-color:rgb(250,200,200)"
    return "color:%s"%c # it's just CSS, so you can do background as well.

In [ ]:
def colour_grouped_table(df):

    return df.style.applymap(highlight_cols, subset=["k_vs_s_%"])


## By Country


In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "date"])

In [ ]:
colour_grouped_table(g)

<<<<<<< local


,country_code,date,s_count,k_count,k_vs_s_%
7,hk,2020-02-16,50190,51343,2.3
8,hk,2020-02-17,52725,53394,1.3
9,hk,2020-02-18,52223,53319,2.1
10,hk,2020-02-19,54526,55459,1.7
11,hk,2020-02-20,51745,52599,1.7
12,hk,2020-02-21,53269,53980,1.3
13,hk,2020-02-22,52155,53100,1.8
28,sg,2020-02-16,329131,346656,5.3
29,sg,2020-02-17,397746,415154,4.4
30,sg,2020-02-18,405034,419638,3.6


,country_code,date,s_count,k_count,is_bot,k_vs_s_%
7,hk,2020-02-09,44568,44669,133,0.2
8,hk,2020-02-10,48563,48796,70,0.5
9,hk,2020-02-11,47089,47508,90,0.9
10,hk,2020-02-12,48352,47896,87,-0.9
11,hk,2020-02-13,54533,54656,145,0.2
12,hk,2020-02-14,47404,48153,96,1.6
13,hk,2020-02-15,52472,53597,127,2.1
28,sg,2020-02-09,294505,302552,348,2.7
29,sg,2020-02-10,309912,317167,415,2.3
30,sg,2020-02-11,312694,325880,363,4.2


>>>>>>> remote


## By Event Type

In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "event_name"])
g.sort_values(["country_code", "event_name"], inplace=True)
colour_grouped_table(g)

<<<<<<< local


,country_code,event_name,s_count,k_count,k_vs_s_%
36,hk,Display user feedback form,5,0,-100
38,hk,LeadGeneration.ClickConversion,745,779,4.6
39,hk,LeadGeneration.ClickedApplyButton,177,194,9.6
40,hk,LeadGeneration.ClickedCTA,3316,3467,4.6
42,hk,LeadGeneration.Conversion,4531,4583,1.1
44,hk,LeadGeneration.FormSubmitted,0,1504,999
46,hk,LeadGeneration.RedirectCompleted,4624,4577,-1
47,hk,LeadGeneration.ShowedMoreDetails,4920,4989,1.4
49,hk,LeadGeneration.ViewedMoreDetails,2661,2757,3.6
51,hk,PageView,168107,173596,3.3


,country_code,event_name,s_count,k_count,is_bot,k_vs_s_%
38,hk,Display user feedback form,5,0,0,-100
40,hk,LeadGeneration.ClickConversion,790,837,0,5.9
41,hk,LeadGeneration.ClickedApplyButton,238,258,0,8.4
42,hk,LeadGeneration.ClickedCTA,2708,2866,0,5.8
44,hk,LeadGeneration.Conversion,4011,4076,0,1.6
46,hk,LeadGeneration.FormSubmitted,243,934,0,284.4
48,hk,LeadGeneration.RedirectCompleted,4243,4237,0,-0.1
49,hk,LeadGeneration.ShowedMoreDetails,6200,6227,0,0.4
51,hk,LeadGeneration.ViewedMoreDetails,2861,2980,0,4.2
54,hk,PageView,155565,158348,708,1.8


>>>>>>> remote


In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta[merged_df_with_meta.event_name=="LeadGeneration.ClickedApplyButton"], ["country_code", "slug_root", "event_name"])

In [ ]:
g

<<<<<<< local


,country_code,slug_root,event_name,s_count,k_count,k_vs_s_%
1,hk,/credit-cards,LeadGeneration.ClickedApplyButton,177.0,194.0,9.6
4,sg,/credit-cards,LeadGeneration.ClickedApplyButton,2.0,2.0,0.0


>>>>>>> remote


<<<<<<< local <removed>


,country_code,slug_root,event_name,s_count,k_count,is_bot,k_vs_s_%
3,hk,/credit-cards,LeadGeneration.ClickedApplyButton,238.0,258.0,0.0,8.4
12,sg,/credit-cards,LeadGeneration.ClickedApplyButton,118.0,123.0,0.0,4.2
13,sg,/debt-consolidation-plan,LeadGeneration.ClickedApplyButton,109.0,117.0,0.0,7.3
14,sg,/personal-loan,LeadGeneration.ClickedApplyButton,30.0,31.0,0.0,3.3


>>>>>>> remote <modified: text/html, text/plain>


## By Top Level Slug

In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta[merged_df_with_meta.page_type!="blog"], ["country_code", "slug_root", "event_name"])
#g.sort_values(["country_code", "slug_root", "event_name"])

#filtering where s_count or k_count is >1000
pv = pd.pivot_table(g[(g.s_count>1000) | (g.k_count>1000)], index=["country_code", "slug_root"], values=["k_count","s_count","k_vs_s_%"], columns=["event_name"], fill_value="")

#colour_grouped_table(pv)
#TODO: not showing up the s_count and k_count :(
pv = pv.swaplevel(0, 1, axis=1).sort_index(axis=1)

In [ ]:
pv.to_excel("kinesis_vs_segment.xlsx")

In [ ]:




#pv[:20].style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, 's-count'])
styled = pv.style.applymap(highlight_cols, subset=pv.columns.get_loc_level('k_vs_s_%', level=1)[0]) #special multi-index on column for colour
#can't get they styling to be happy with colors
#styled.to_excel("kinesis_vs_segment.xlsx", engine='openpyxl') #use special engine for formatting https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

styled

<<<<<<< local


>>>>>>> remote


In [ ]:
#pv.to_html("kinesis_vs_segment.html")

## By Type of Page

In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "page_type"])
colour_grouped_table(g)

<<<<<<< local


,country_code,page_type,s_count,k_count,k_vs_s_%
5,hk,blog,232080,233607,0.7
6,hk,iss,18386,18460,0.4
7,hk,lps,4422,4497,1.7
8,hk,shop,111945,116630,4.2
20,sg,blog,1.48473e+06,1.51691e+06,2.2
21,sg,iss,36082,35183,-2.5
22,sg,lps,6110,6334,3.7
23,sg,shop,830735,907890,9.3
24,sg,unbounce,15112,15647,3.5


>>>>>>> remote


<<<<<<< local <removed>


,country_code,page_type,s_count,k_count,is_bot,k_vs_s_%
5,hk,blog,214085,215184,607,0.5
6,hk,iss,16452,16538,0,0.5
7,hk,lps,4271,4409,6,3.2
8,hk,shop,108173,109144,135,0.9
20,sg,blog,1.30896e+06,1.33945e+06,979,2.3
21,sg,iss,50882,50168,6,-1.4
22,sg,lps,4092,4306,8,5.2
23,sg,shop,722200,769458,1548,6.5
24,sg,unbounce,6619,6843,0,3.4


>>>>>>> remote <modified: text/html, text/plain>


### Type of Page, Just Pageviews

In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta[(merged_df_with_meta.event_name=="PageView") & (merged_df_with_meta.country_code.isin(["sg", "hk"]))], ["country_code", "page_type"])
colour_grouped_table(g)

<<<<<<< local <modified: text/html, text/plain>


,country_code,page_type,s_count,k_count,k_vs_s_%
5,hk,blog,78206,79050,1.1
6,hk,iss,9231,9300,0.7
7,hk,lps,3824,3849,0.7
8,hk,shop,76846,81397,5.9
20,sg,blog,527846,545490,3.3
21,sg,iss,17739,18274,3
22,sg,lps,5610,5777,3
23,sg,shop,467348,509792,9.1
24,sg,unbounce,15112,15647,3.5


,country_code,page_type,s_count,k_count,is_bot,k_vs_s_%
5,hk,blog,72072,72951,586,1.2
6,hk,iss,8198,8225,0,0.3
7,hk,lps,3656,3720,6,1.8
8,hk,shop,71639,73452,116,2.5
20,sg,blog,480957,497781,946,3.5
21,sg,iss,24351,24798,2,1.8
22,sg,lps,3729,3889,8,4.3
23,sg,shop,407195,433442,1420,6.4
24,sg,unbounce,6619,6843,0,3.4


>>>>>>> remote <modified: text/html, text/plain>


### By Device Type and Country for pageviews in HK and SG

In [ ]:
merged_df_with_meta.head()

# By Device Type

## Pageviews by Device Type and OS

In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta[(merged_df_with_meta.event_name=="PageView") \
                                                           & (merged_df_with_meta.country_code.isin(["sg", "hk"]))\
                                                          # & ((merged_df_with_meta.s_count>100) | (merged_df_with_meta.k_count>100))
                                                          ]\
                                       , ["device_family", "os_family"])
colour_grouped_table(g)

## All Events Desktop and mobile by browser, popular ones

In [ ]:
browser_popularity_threshold = len(merged_df_with_meta)/100 #must have 1% of traffic to be interesting.
g = group_by_and_show_count_difference(merged_df_with_meta[(merged_df_with_meta.device_family.isin(["mobile", "desktop"])) \
                                                           & (merged_df_with_meta.country_code.isin(["sg", "hk"]))\
                                                          # & ((merged_df_with_meta.s_count>100) | (merged_df_with_meta.k_count>100))
                                                          ]\
                                       , ["device_family", "os_family", "browser_family"])
g = g[(g.s_count > browser_popularity_threshold) |(g.k_count>browser_popularity_threshold)]
#remainder = g[(g.s_count <= browser_popularity_threshold) |(g.k_count<=browser_popularity_threshold)]
#print("%i events (%.3f percent of total) were in smaller browsers"%(len(remainder), len(remainder)/len(merged_df_with_meta)))
colour_grouped_table(g)

## Anonymous_ids with big differences 
(might be bots or duplicates or whatever)

In [ ]:
# This has non-standard code because the standard code was taking too long to run and running out of memory.
cols_to_group_by = ["anonymous_id", "device_family", "os_family", "browser_family"]
# g = group_by_and_show_count_difference(merged_df_with_meta[ (merged_df_with_meta.country_code.isin(["sg", "hk"]))], cols_to_group_by)
g = merged_df_with_meta[merged_df_with_meta.country_code.isin(["sg", "hk"])].groupby(cols_to_group_by, observed=True).sum().reset_index() #need to use observed or it creates loads of empty rows and runs out of memory
# g = g[(g["k_vs_s_%"].abs()>50) & ((g.k_count>20) | (g.s_count>20))]

#colour_grouped_table(g)

In [ ]:
unique_pages_per_anon_id = merged_df_with_meta.groupby('anonymous_id').page_url.nunique()

In [ ]:
len(g)

In [ ]:
len(unique_pages_per_anon_id)

In [ ]:
g=g.merge(unique_pages_per_anon_id, left_on="anonymous_id", right_on="anonymous_id")
g.rename(columns={"page_url":"num_pages"}, inplace=True)

In [ ]:
g.head(5)

In [ ]:
len(merged_df_with_meta)

<<<<<<< local


2008810

1794946

>>>>>>> remote


In [ ]:
big_one_threshold = 50
big_ones = g[((g.k_count>big_one_threshold) | (g.s_count>big_one_threshold)) & (((g.k_count - g.s_count)/g.s_count).abs()>0.5)]  
#big_ones.sort_values(big_ones.s_count)

#g.sort_values("k_vs_s_%")
#colour_grouped_table(g)
big_ones_sorted = big_ones.sort_values("k_count", ascending = False)
def highlight_big_num_pages(x):
    threshold_num = 20
    if x<threshold_num:
        return ""
    else:
        c_value = int(min(255, (x-threshold_num)/80*255))
        return "background-color:rgb(%i, %i, %i)"%(255, 255-c_value, 255-c_value)
    
big_ones_sorted.style.applymap(highlight_big_num_pages, subset=["num_pages"])


In [ ]:
# dig into the top one
top_anon_id = big_ones.sort_values("k_count", ascending = False)["anonymous_id"].values[0]
top_anon_df = merged_df_with_meta[merged_df_with_meta.anonymous_id==top_anon_id]
print("looking into %s \n %s"% (top_anon_id, top_anon_df["user_agent"].values[0]))


looking into 8cce5789-b10f-4fda-934b-935b8356dd29 
 Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexus 5 Build/JOP40D) AppleWebKit/535.19 (KHTML, like Gecko; googleweblight) Chrome/38.0.1025.166 Mobile Safari/535.19


In [ ]:
top_anon_df.groupby("user_agent").sum()

In [ ]:
g = group_by_and_show_count_difference(top_anon_df, ["event_name"])
colour_grouped_table(g)

In [ ]:
<<<<<<< local
# g = group_by_and_show_count_difference(top_anon_df, ["page_url"])
# colour_grouped_table(g)
=======
g = group_by_and_show_count_difference(top_anon_df, ["page_url"])
colour_grouped_table(g)
>>>>>>> remote

In [119]:
top_anon_df.groupby(["page_type"]).sum()

,s_count,k_count
page_type,,
blog,71,6614
iss,0,0
lps,0,0
shop,9,199
unbounce,0,0


# Possible bots

In [ ]:
anonymous_ids_counts = group_by_and_show_count_difference(merged_df_with_meta, ["anonymous_id"])>>>>>

# Other Issues to Check For
* duplicates
* skipping "https" from the url (observed as a current issue)
* certain browsers having issues
* users creating a lot of duplicate events (and doing the above analysis using sum vs count)

# Play Space

In [ ]:
blog_pageviews = merged_df_with_meta[(merged_df_with_meta.event_name=="PageView") & (merged_df_with_meta.country_code.isin(["sg", "hk"])) & (merged_df_with_meta.page_type=="blog")]

In [ ]:
blog_pageviews.groupby(["country_code", "date"]).size().reset_index()

In [111]:

g = group_by_and_show_count_difference(merged_df_with_meta[(merged_df_with_meta.page_type=="iss")  & (merged_df_with_meta.country_code.isin(["sg", "hk"])) & (merged_df_with_meta.page_url.str.contains("iss.moneysmart")) & (merged_df_with_meta.page_url.str.contains("/credit-card"))] , ["country_code", "event_name", "slug"])
#g = g[(g.s_count > browser_popularity_threshold) |(g.k_count>browser_popularity_threshold)]
#remainder = g[(g.s_count <= browser_popularity_threshold) |(g.k_count<=browser_popularity_threshold)]
#print("%i events (%.3f percent of total) were in smaller browsers"%(len(remainder), len(remainder)/len(merged_df_with_meta)))
colour_grouped_table(g)

,country_code,event_name,slug,s_count,k_count,k_vs_s_%
21310,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/aeon-card-jal/redirect,3,1,-66.7
21324,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/american-express-cathay-pacific-credit-card/redirect,1,1,0
21327,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/american-express-cathay-pacific-elite-credit-card/redirect,0,1,999
21330,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/american-express-platinum-credit-card/redirect,4,3,-25
21389,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/citi-cash-back-visa-card/redirect,2,2,0
21393,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/citi-prestige-card/redirect,2,2,0
21397,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/citibank-premiermiles-card/redirect,8,5,-37.5
21400,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/citibank-rewards-visa-card/redirect,10,10,0
21420,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/dah-sing-british-airways-platinum-card/redirect,1,0,-100
21422,hk,LeadGeneration.RedirectCompleted,/en/credit-cards/dah-sing-one-credit-card/redirect,2,1,-50


In [112]:
g = group_by_and_show_count_difference(merged_df_with_meta[(merged_df_with_meta.page_type=="iss")  & (merged_df_with_meta.country_code.isin(["sg", "hk"])) & (merged_df_with_meta.page_url.str.contains("iss.moneysmart")) ] , ["country_code", "event_name", "slug_root"])
colour_grouped_table(g)

,country_code,event_name,slug_root,s_count,k_count,k_vs_s_%
14,hk,LeadGeneration.RedirectCompleted,/credit-cards,293,184,-37.2
18,hk,LeadGeneration.RedirectCompleted,/personal-loan,40,19,-52.5
21,hk,PageView,/credit-cards,376,397,5.6
25,hk,PageView,/personal-loan,51,56,9.8
56,sg,LeadGeneration.RedirectCompleted,/credit-cards,6251,5182,-17.1
57,sg,LeadGeneration.RedirectCompleted,/credit_cards,3,3,0
58,sg,LeadGeneration.RedirectCompleted,/debt-consolidation-plan,192,160,-16.7
59,sg,LeadGeneration.RedirectCompleted,/debt_consolidation_plan,1,1,0
60,sg,LeadGeneration.RedirectCompleted,/personal-loan,2187,1800,-17.7
61,sg,LeadGeneration.RedirectCompleted,/travel-insurance,658,597,-9.3


In [113]:
g = group_by_and_show_count_difference(merged_df_with_meta[(merged_df_with_meta.page_type=="iss")  & (merged_df_with_meta.country_code.isin(["sg", "hk"])) & (merged_df_with_meta.event_name=="PageView") ] , ["country_code", "event_name", "slug_root", "date"])
colour_grouped_table(g)

,country_code,event_name,slug_root,date,s_count,k_count,k_vs_s_%
140,hk,PageView,/banks-loan,2020-02-16,42,42,0
141,hk,PageView,/banks-loan,2020-02-17,54,50,-7.4
142,hk,PageView,/banks-loan,2020-02-18,52,53,1.9
143,hk,PageView,/banks-loan,2020-02-19,56,56,0
144,hk,PageView,/banks-loan,2020-02-20,36,34,-5.6
145,hk,PageView,/banks-loan,2020-02-21,42,46,9.5
146,hk,PageView,/banks-loan,2020-02-22,46,46,0
154,hk,PageView,/credit-cards,2020-02-16,106,107,0.9
155,hk,PageView,/credit-cards,2020-02-17,137,138,0.7
156,hk,PageView,/credit-cards,2020-02-18,140,145,3.6


In [114]:
merged_df_with_meta.columns

Index(['anonymous_id', 'event_name', 'page_url', 'date', 's_count', 'k_count',
       'page_type', 'slug', 'slug_root', 'ab_test', 'country_code',
       'user_agent', 'device_family', 'os_family', 'os_version',
       'browser_family', 'browser_version', 'is_bot'],
      dtype='object')